In [14]:
import requests
import pandas as pd
import json
import ast
import yaml

# Before you can connect the Twitter API, 
# you’ll need to set up the URL to ensure it has the right fields so you get the right data back. 
def create_twitter_url(handle):
    # handle = "patagonia"
    max_results = 100
    mrf = "max_results={}".format(max_results)
    q = "query=from:{}".format(handle)
    url = "https://api.twitter.com/2/tweets/search/recent?tweet.fields=lang&{}&{}".format(mrf, q)
    return url

# To access the configuration file you created while setting up config.yaml
def process_yaml():
    with open("config.yaml") as file:
        return yaml.safe_load(file)

# To access the bearer token from your config.yaml file
def create_bearer_token(data):
    return data["search_tweets_v2"]["bearer_token"]


# To format the headers to pass in your bearer_token and url
def twitter_auth_and_connect(bearer_token, url):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    response = requests.request("GET", url, headers=headers)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

def no_tweets(res_json):
    if res_json == {"meta": {"result_count": 0}}:
        print("The Twitter handle entered hasn't Tweeted in 7 days.")


def connect_to_azure(data):
    azure_url = "https://senti-on-tweet.cognitiveservices.azure.com/"
    sentiment_url = "{}text/analytics/v2.1/sentiment".format(azure_url)
    subscription_key = data["azure"]["subscription_key"]
    return sentiment_url, subscription_key


def azure_header(subscription_key):
    return {"Ocp-Apim-Subscription-Key": subscription_key}


def create_document_format(res_json):
    data_only = res_json["data"]
    doc_start = '"documents": {}'.format(data_only)
    str_json = "{" + doc_start + "}"
    dump_doc = json.dumps(str_json)
    doc = json.loads(dump_doc)
    return ast.literal_eval(doc)


def sentiment_scores(headers, sentiment_url, document_format):
    response = requests.post(sentiment_url, headers=headers, json=document_format)
    return response.json()


def mean_score(sentiments):
    sentiment_df = pd.DataFrame(sentiments["documents"])
    return sentiment_df["score"].mean()


def week_logic(week_score):
    if week_score > 0.75 or week_score == 0.75:
        print("You had a positve week")
    elif week_score > 0.45 or week_score == 0.45:
        print("You had a neautral week")
    else:
        print("You had a negative week, I hope it gets better")

'''
def main():
    url = create_twitter_url()
    data = process_yaml()
    bearer_token = create_bearer_token(data)
    res_json = twitter_auth_and_connect(bearer_token, url)
    no_tweets(res_json)
    sentiment_url, subscription_key = connect_to_azure(data)
    headers = azure_header(subscription_key)
    document_format = create_document_format(res_json)
    sentiments = sentiment_scores(headers, sentiment_url, document_format)
    week_score = mean_score(sentiments)
    print(week_score)
    week_logic(week_score)


if __name__ == "__main__":
    main()
'''
def analysis(handle):
    url = create_twitter_url(handle)
    data = process_yaml()
    bearer_token = create_bearer_token(data)
    res_json = twitter_auth_and_connect(bearer_token, url)
    no_tweets(res_json)
    sentiment_url, subscription_key = connect_to_azure(data)
    headers = azure_header(subscription_key)
    document_format = create_document_format(res_json)
    sentiments = sentiment_scores(headers, sentiment_url, document_format)
    week_score = mean_score(sentiments)
    print(week_score)
    week_logic(week_score)
    return pd.DataFrame.from_dict(res_json['data'])

In [15]:
pd.set_option('display.max_colwidth', None)

In [16]:
# Pull out all the tweets for Patagonia in last 7 days
patagonia = analysis("patagonia")
patagonia

0.7974811989527483
You had a positve week


,id,lang,text
0,1425936234984050689,en,"If it’s going to work as hard as you do, it’s got to have hemp. Durable, breathable and broken-in off the shelf, our full line of industrial hemp Workwear keeps you covered and focused on the job at hand."
1,1425936181796106242,en,"If it’s going to work as hard as you do, it’s got to have hemp. Durable, breathable and broken-in off the shelf, our full line of industrial hemp Workwear keeps you covered and focused on the job at hand."
2,1425924934673469442,en,@Joshua7Berry Hey Joshua! We currently have inventory of the M's Lightweight Synchilla Snap-T Fleece Pullover in Oatmeal Heather online https://t.co/Ts4OBvGIo5. Feel free to DM us with specific questions! https://t.co/BQQTdfLPtb
3,1425911008640466946,en,"After 40 years of supporting families by offering onsite childcare, we’ve learned we can create a more equitable and productive culture, help retain and attract top talent and recoup most of the cost. Thanks for creating and leading on a national family first policy agenda @VP."
4,1425909567750213633,en,Every fleece is Fair Trade Certified™ sewn and made of durable 100% recycled polyester fleece to keep your little ones warm when the temperature drops.
...,...,...,...
60,1423731380500975617,en,Join Patagonia grantee @SalmonStateAK on August 10th to celebrate and share stories about Alaska's salmon way of life. Click to learn more and RSVP.
61,1423725144678219777,en,Join Patagonia grantee @NRCMenvironment in calling on decision-makers to remove the Shawmut Dam and restore access to critical habitat for endangered Atlantic salmon. Add your voice by August 14th.
62,1423711121396150274,en,"Join us for a conversation with activists and artists and learn how to volunteer your professional skills with grassroots environmental groups. Sign up to join the event on Thursday, August 10: https://t.co/L7AFrqBYE2 https://t.co/oA56nxoU5h"
63,1423672379537072128,en,"""The public needs to urge @POTUS and Congress to exclude any new investments in dams in the 21st Century Dams Act and infrastructure package — except for urgent safety matters — and instead use the legislation to fund dam removal.""\nhttps://t.co/BhwT4V0qV1"


In [17]:
text_patagonia = patagonia.drop(['id', 'lang'], axis=1).drop_duplicates('text')
text_patagonia.count()

text    47
dtype: int64

In [18]:
retweets_patagonia = text_patagonia[text_patagonia['text'].str.contains('RT')]
retweets_patagonia

,text
27,"RT @WHCEQ: Today, we launched the #AmericatheBeautiful Interagency Working Group. Chaired by @BrendaMallory46, @SecDebHaaland, @SecVilsack…"
28,"RT @GrandCanynTrust: Tonight 6PM MDT, join Janene Yazzie, June Lorenzo, and Brett Lee Shelton for a discussion of free, prior, and informed…"
36,RT @conservationall: Join us TOMORROW at #ORshow for our breakfast event at the Colorado Convention Center (7am MT) or tune in virtually vi…


In [22]:
Social_Good = text_patagonia[text_patagonia['text'].str.contains('protect|help|environment')]
print(Social_Good.count())
Social_Good

text    18
dtype: int64


,text
3,"After 40 years of supporting families by offering onsite childcare, we’ve learned we can create a more equitable and productive culture, help retain and attract top talent and recoup most of the cost. Thanks for creating and leading on a national family first policy agenda @VP."
18,Take action with Patagonia grantee @WFA_WildFarm as they work to ensure that organic standard rules include the protection of native ecosystems. Click to add your name.
19,Join Patagonia grantee @WFA_WildFarm in calling on the National Organic Program to ensure that organic standard rules include the protection of native ecosystems. Click to add your name.
24,Join Patagonia grantee @NRCM in calling on Maine leaders to support permanent protection for the Arctic National Wildlife Refuge. Click to take action.
25,Join Patagonia grantee @NRCM in calling on Maine leaders to help permanently protect the Arctic Refuge from oil and gas drilling.
26,Join Patagonia grantee @NRCM in calling on Maine leaders to help permanently protect the Arctic National Wildlife Refuge from oil and gas drilling.
29,Join Patagonia grantee @NRCM in calling on Maine leaders to support permanent protections for the Arctic Refuge. Add your voice to defend this ecologically and culturally significant region from fossil fuel drilling.
30,Patagonia grantee @GrandCanynTrust is working to support long-term safeguards for the Little Colorado River near the Grand Canyon. Read their blog post to learn how they helped fend off proposals to dam the river and next steps for protecting this iconic region.
31,Patagonia grantee @GrandCanynTrust is working to support long-term safeguards for the Little Colorado River near the Grand Canyon. Read their blog post to learn how they helped fend off two proposals to dam the river and next steps for protecting this iconic region.
32,Patagonia grantee @GrandCanyonTrust is working to support long-term safeguards for the Little Colorado River near the Grand Canyon. Read their blog post to learn how they helped fend off two proposals to dam the river and next steps for protecting this iconic region.


In [23]:
reply = text_patagonia[text_patagonia['text'].str.contains('Hi|Thank you|Hey')]
print(reply.count())
reply

text    8
dtype: int64


,text
2,@Joshua7Berry Hey Joshua! We currently have inventory of the M's Lightweight Synchilla Snap-T Fleece Pullover in Oatmeal Heather online https://t.co/Ts4OBvGIo5. Feel free to DM us with specific questions! https://t.co/BQQTdfLPtb
35,"@me_idealist Hey there, this is the address: 189 Stag Hill Rd, Mahwah, NJ 07430-1041"
43,"Sam DeNicola doesn’t consider himself a great baker or a cyclist. So why did he join his friends to start Bread Bike, a bicycle-delivery breadmaking business in San Luis Obispo, California? His reasons are deliciously unexpected."
50,"@megmegmehg Hi Meg, these are two different styles of Snap-T. Our Lightweight Synchilla® Snap-T® Fleece Pullover is heavier than our Micro D® Snap-T® Fleece Pullover making the Lightweight a higher price point. Men's and women's gear that are the same style will always be the same price."
51,"@robareback Hi Robare, we do still carry the Men's Stretch Wavefarer Walk Shorts, but we have completely sold out for the Spring 2021 season. Try checking out our Worn Wear Patagonia website, sometimes the Men's Stretch Wavefarer Walk Shorts will show up there, https://t.co/Z2mnz44ZyG."
53,@steveyoungburns Hey there! We're happy to go over garment dimensions with you. Send us a DM and we'll talk more. https://t.co/BQQTdfLPtb
56,"@Kaffe_Takk Hi there, we'd be happy to help you out with some care instructions. Please reach out to us privately so we can further assist you. https://t.co/BQQTdfLPtb"
57,@LoriSaldanaSD Oh my! Thank you so much for pointing this out us. We'll let the appropriate members of our team know about this.


In [25]:
# Pull out all the tweets for JanSport in last 7 days
JanSport= analysis("JanSport")
JanSport

The Twitter handle entered hasn't Tweeted in 7 days.


KeyError: 'data'

In [28]:
columbia = analysis("Columbia1938")
columbia

0.993035078048706
You had a positve week


,id,lang,text
0,1425215856708132870,en,"To help you make the right choice, we’ve listed seven of the most important qualities to keep in mind when you’re shopping for a backpack. \nhttps://t.co/P1ByHRq3Fu"


In [29]:
puma = analysis("PUMA")
puma

0.6061198830604553
You had a neautral week


,id,lang,text
0,1425873053259534339,en,PUMA FAM BRINGING HOME THOSE MEDALS. #OnlySeeGreat https://t.co/dlj3EkUkpf
1,1425819369456640002,en,🖤🤍 @dannapaola in Mayze\nhttps://t.co/MsygWDlljW https://t.co/lk2ufCCVOv
2,1425522332886437891,en,RT @Bratz: Thursday! 👄👟 @PUMA #bratz https://t.co/vdGDnOExSH
3,1425500387763757059,en,RT @Babyyhairz: The Hussle Way \n“Miami Story” August 13\n@PUMA x @themarathonclothing https://t.co/zuWagsy0Yk
4,1425077221518098437,und,RT @MELOD1P: MB.01
5,1425077213179809793,en,RT @brkicks: First look at LaMelo Ball’s first signature shoe with Puma called the MB1 😮 @NickDePaula https://t.co/AKzCYxdh6l
6,1425055103938174977,en,RT @NickDePaula: The Puma Jet — the brand’s private plane for its athletes — is still one of the best endorsement perks out. \n\nHere’s how L…
7,1424732255461707779,en,PU(MA)LISIC 💨💨💨 \n\nWelcome to the FAM @cpulisic_10 😍 https://t.co/rE2VVBWt1j
8,1423789183215284224,und,#OnlySeeGreat @ByGollyMolly12 https://t.co/yppWxPohOm
9,1423661050797428740,und,Меняй бег с новой экипировкой PUMA Spectra Run с технологиями DRYcell и COOLadapt 🙌


In [34]:
text_puma = puma[puma['lang']=='en'].drop(['id', 'lang'], axis=1).drop_duplicates('text')
text_puma.count()
text_puma

,text
0,PUMA FAM BRINGING HOME THOSE MEDALS. #OnlySeeGreat https://t.co/dlj3EkUkpf
1,🖤🤍 @dannapaola in Mayze\nhttps://t.co/MsygWDlljW https://t.co/lk2ufCCVOv
2,RT @Bratz: Thursday! 👄👟 @PUMA #bratz https://t.co/vdGDnOExSH
3,RT @Babyyhairz: The Hussle Way \n“Miami Story” August 13\n@PUMA x @themarathonclothing https://t.co/zuWagsy0Yk
5,RT @brkicks: First look at LaMelo Ball’s first signature shoe with Puma called the MB1 😮 @NickDePaula https://t.co/AKzCYxdh6l
6,RT @NickDePaula: The Puma Jet — the brand’s private plane for its athletes — is still one of the best endorsement perks out. \n\nHere’s how L…
7,PU(MA)LISIC 💨💨💨 \n\nWelcome to the FAM @cpulisic_10 😍 https://t.co/rE2VVBWt1j


In [33]:
retweets_puma = text_puma[text_puma['text'].str.contains('RT')]
retweets_puma

,text
2,RT @Bratz: Thursday! 👄👟 @PUMA #bratz https://t.co/vdGDnOExSH
3,RT @Babyyhairz: The Hussle Way \n“Miami Story” August 13\n@PUMA x @themarathonclothing https://t.co/zuWagsy0Yk
5,RT @brkicks: First look at LaMelo Ball’s first signature shoe with Puma called the MB1 😮 @NickDePaula https://t.co/AKzCYxdh6l
6,RT @NickDePaula: The Puma Jet — the brand’s private plane for its athletes — is still one of the best endorsement perks out. \n\nHere’s how L…
